### 투빅스 15기 강지우 


#### TOBIG'S 14기 정규세션 4주차 SVM 
### ASSIGNMENT1. Multiclass SVM 구현

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [4]:
scal = StandardScaler() #scaling
X = scal.fit_transform(X)

In [5]:
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [6]:
# One VS Rest 구현을 위해 1) class가 0 or not 2)class가 1 or not을 구분하기 위한 머신 등을 미리 만들어주자
svm_1 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_2 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_3 = SVC(kernel ='rbf', C = 5, gamma = 5)

In [7]:
from sklearn.model_selection import train_test_split #test/train 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [10]:
y_test.shape

(30,)

In [8]:
y_train = pd.get_dummies(y_train) #one hot encoding

In [9]:
svm_1.fit(X_train,y_train.iloc[:,0]) # 데이터 클레스가 0 or not 구분해주는 머신
svm_2.fit(X_train,y_train.iloc[:,1]) # 데이터 클레스가 1 or not 구분해주는 머신
svm_3.fit(X_train,y_train.iloc[:,2]) # 데이터 클레스가 2 or not 구분해주는 머신
print(svm_1.predict(X_test)) #데이터 클래스가 0 or not을 구분해주는 애를 통해서 테스트 데이터의 클래스가 0인지, 0이 아닌인지 예측해보자

print(svm_1.decision_function(X_test)) #decision_function hyperplane과의 거리를 구하는 방법(필요하다면 활용해주세요!)

[0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0]
[-1.12470845 -0.86326953 -0.65281099 -0.50248821 -0.76284323 -0.87465573
  1.07709345 -0.99281647  0.47441336 -0.99842743 -0.83989348  0.15633457
  0.32871788 -0.97965464 -0.72385083 -0.92638376  1.28322481 -0.56240455
 -0.72719892 -0.99509775  0.43166724 -0.96451557 -0.82991366 -1.03020581
 -0.75166835  1.13461335  0.39943974 -1.04194106 -0.93376548 -1.06133798]


In [34]:
svm_1.predict(X_test)

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0], dtype=uint8)

In [51]:
# 부호가 모든 같은 경우가 있는가? > 모두 동점인 경우가 생길 것
for i in range(len(X_test)):
    # ~. decision_function을 이용하면 해당 데이터가 하이퍼플레인으로부터 얼마나 떨어져있는지 '거리'가 나온다!
    # 다음은 그 값의 부호를 이용해 모두가 동점인 경우가 있는지 출력하는 함수 
    if( np.sign(svm_1.decision_function(X_test)[i]) == np.sign(svm_2.decision_function(X_test)[i])) and (np.sign(svm_2.decision_function(X_test)[i]) == np.sign(svm_3.decision_function(X_test)[i])):
        print(i)

3
17
18


In [48]:
## Case 1 : One vs Rest SVM을 이부분에 구현해주세요 위 결과들을 이용해서 multi class SVM을 직접 구현해주세요! 하드코딩이 하시기 편할겁니다.
from sklearn.metrics import accuracy_score

def one_rest_svm(i):
    label = ['setosa','versicolor','virginica']
    # 각 row 마다 예측값을 저장 
    pred = []
    pred.append(svm_1.predict(X_test)[i])
    pred.append(svm_2.predict(X_test)[i])
    pred.append(svm_3.predict(X_test)[i])
    
    # 복수의 결과가 존재할 시 Hyperplayne을 기준으로 가장 가까운 거리를 가지는 범주로 예측 
    if sum(pred) > 2  or sum(pred) < 1: 
        distances = []
        distances.append(svm_1.decision_function(X_test)[i])
        distances.append(svm_2.decision_function(X_test)[i])
        distances.append(svm_3.decision_function(X_test)[i])
        
        if sum(pred) > 2:
            distances_ = [i if i > 0 else 999 for i in distances]
            return label[distances_.index(min(distances_))]
        elif sum(pred) < 1:
            return label[distances.index(min(distances))]
    
    # 하나의 결과가 존재할 시에는 그대로 해당 범주로 예측
    else:
        return label[pred.index(1)]

# 위의 함수로 predict
preds = []
for i in range(len(X_test)):
    preds.append(one_rest_svm(i))
one_rest_svm_acc = accuracy_score(y_test, preds)
print('one_rest_svm_acc:', one_rest_svm_acc)

one_rest_svm_acc: 0.8666666666666667


In [54]:
# 원래 라이브러리가 제공하는 multi class SVM과 여러분이 구현한 multiclass SVM 결과를 비교해주세요
from sklearn.model_selection import train_test_split #데이터셋 분리
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=48)

# 커널 사용 svm
svm_4 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_4.fit(X_train_2, y_train_2)
y_pred = svm_4.predict(X_test_2)

print('using kernel:', accuracy_score(y_test_2,y_pred))

# 커널 미사용 svm
svm_5 = SVC()
svm_5.fit(X_train_2, y_train_2)
y_pred = svm_5.predict(X_test_2)
print('not using kernel:', accuracy_score(y_test_2,y_pred))

using kernel: 0.8333333333333334
not using kernel: 0.9333333333333333
